### Super Store Sales Analysis and Profit Return Prediction

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://images.unsplash.com/photo-1630342654882-aed9e2e86f24?q=80&w=1974&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D", width=1300, height=500)

### Introduction

This Project looks to explore a dataset containing information on a small retail business

Objectives include:

1. Analyse the sales data and pinpoint areas and opportunities for the business to experience a growth boost.

    This would be done by providing insights and answers to particular business questions tailored to the data on board.

        1.  Best Selling Category
        2.  Most Profitable Category
        3.  Best Selling Sub-Category
        4.  Most Profitable Sub-Category
        5.  Most Profitable Regions and Cities
        6.  Overall Sales (Over The Years)
        7.  Most Selling and Profitable Products

2. Next, We will carry out some machine learning on the dataset, in an attempt to predict profit returns based of information available in a record.

    Model would be a regression model but would return a final output of either "Green" or "Red".
    
        "Green" being that the model predicts a profit return of 15% or above.
        "Red" being that the model predicts a profit return less than 15%. 

### Environment Preparation

In [ ]:
# Importing libraries
# Importing alias for easy reading

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

# Reading data from csv
df = pd.read_csv('Sample - Superstore.csv')

# Data Exploration

In [ ]:
# Preview first 5 rows of data set
df.head()

In [ ]:
# Preview last 5 rows of data set
df.tail()

In [ ]:
df = df.drop(df.columns[:2], axis=1)

df.shape

In [ ]:
df.info()

## Data Cleaning

### First Stage - Handling Duplicate Records

In [ ]:
# Find the number of duplicate data
df.duplicated().sum()

### Second Stage - Handling Missing Data

In [ ]:
# To get the number of records with null values...

df.isnull().sum()

In [ ]:
# Visualizing the distribution of missing data
msno.matrix(df)

##### Mean Imputation For Numeric Features and (Most Frequent) Imputation for Categorical Features

In [ ]:
# Identifing categorical and numeric columns
categorical_cols = df.select_dtypes(include='object')
numeric_cols = df.select_dtypes(include='number')

# Imputing missing values for numeric columns with the mean
df[numeric_cols.columns] = df[numeric_cols.columns].fillna(numeric_cols.mean())

# Imputing missing values for categorical columns with the mode (most frequent value)
for col in categorical_cols.columns:
    mode_value = df[col].mode()[0]
    df[col] = df[col].fillna(mode_value)

In [ ]:
# Re-visualizing the distribution of missing data
msno.matrix(df)

In [ ]:
# To get the number of records with null values...

df.isnull().sum()

### Feature Engineering - Adding a Column for Profit Return Percentage

In [ ]:
df ['Profit Percentage'] = (df.Profit / df.Sales) * 100
df ['Cost Price'] = (df.Sales - df.Profit)

df.head()

### Descriptive Analysis

In [ ]:
df.describe(include = "all")

In [ ]:
# To read the data as a time series, we have to pass special arguments to the read_csv command:
from datetime import datetime
dateparse = lambda x: datetime.strptime(x, '%d/%m/%Y')

In [ ]:
#Converting Order Date to Datetime
df['Order Date']= pd.to_datetime(df['Order Date'])

#Creating a Month Year Column
df['Year Month'] = df['Order Date'].apply(lambda x: x.strftime('%Y-%m'))

In [ ]:
sales_trend = pd.DataFrame(df.groupby('Year Month')['Sales'].sum().reset_index())
sales_trend.head()

In [ ]:
# Data
x = sales_trend['Year Month']
y = sales_trend['Sales']

# Plotting
plt.figure(figsize=(20, 5))
plt.xticks(rotation='vertical', size=10)
plt.plot(x, y, color='red')
plt.fill_between(x, y, color='red', alpha=0.2)
plt.title('SALES TREND (2014 - 2017)')
plt.ylabel('Sales')
plt.xlabel('Year Month')
plt.show()

### Discoveries

### Exploratory Data Analysis

Remember when we said we would provide insights and answers to certain business questions.

First, We would explore the 'Categories'

    1. Sales
    2. Profit
    3. Quantity

In [ ]:
# Grouping Sales, Profit and Quantity by Category

category_analysis = pd.DataFrame(df.groupby(['Category']) [['Sales', 'Profit', 'Quantity']].sum())

category_analysis.head()

##### We'd Get These on Graphs For easier Visualization

In [ ]:
category_analysis.plot(kind='bar', subplots=True, layout=(1,3), figsize=(15,3))
plt.show()

### We would do the same for 'Sub Categories'

In [ ]:
# Grouping Sales, Profit and Quantity by Sub-Sub_Category

Sub_Category_analysis = pd.DataFrame(df.groupby(['Sub-Category']) [['Sales', 'Profit', 'Quantity']].sum())

Sub_Category_analysis_Sales = Sub_Category_analysis.sort_values('Sales', ascending=False)

Sub_Category_analysis_Sales.head(30)

In [ ]:
Sales_SubCategory = pd.DataFrame(df.groupby(['Sub-Category']) [['Sales']].sum().sort_values('Sales', ascending=False))

Sales_SubCategory.head()

Sales_SubCategory.plot(kind='bar', subplots=True, figsize=(15,3), color=['green'])
plt.show()

In [ ]:
Profit_SubCategory = pd.DataFrame(df.groupby(['Sub-Category']) [['Profit']].sum().sort_values('Profit', ascending=False))

Profit_SubCategory.head()

Profit_SubCategory.plot(kind='bar', subplots=True, figsize=(15,3), color=['blue'])
plt.show()

In [ ]:
Quantity_SubCategory = pd.DataFrame(df.groupby(['Sub-Category']) [['Quantity']].sum().sort_values('Quantity', ascending=False))

Quantity_SubCategory.head()

Quantity_SubCategory.plot(kind='bar', subplots=True, figsize=(15,3), color=['brown'])
plt.show()

In [ ]:
# Grouping Sales, Profit and Quantity by Cities

City_analysis = pd.DataFrame(df.groupby(['City']) [['Sales', 'Profit', 'Quantity']].sum().sort_values('Sales', ascending= False))

Top_Cities = City_analysis[:10]
Bottom_Cities = City_analysis.tail(10)

Top_Cities.head()

### Let's take a look at Cities with the Highest and lowest Sales, Profit, and Quantity Sold in

### Top Cities

In [ ]:
Top_Cities.plot(kind='bar', subplots=True, layout=(1,3), figsize=(15,3), color=['red', 'green', 'blue'])
plt.show()

### Bottom Cities

In [ ]:
Bottom_Cities.plot(kind='bar', subplots=True, layout=(1,3), figsize=(15,3), color=['red', 'green', 'blue'])
plt.show()

In [ ]:
import matplotlib.pyplot as plt

Cities = pd.concat([Top_Cities, Bottom_Cities])
diff = Cities['Sales'] - Cities['Profit']
colors = ['red' if x < 0 else 'green' for x in diff]

ax = diff.plot(kind='barh', figsize=(10,6), color=colors)
ax.set_xlabel('Difference')
ax.set_ylabel('City')
ax.set_title('Difference between Sales and Profit for Top and Bottom Cities')
plt.show()


In [ ]:
df.to_csv('data.csv', index=False)